In [9]:
class_file = "truncated_classes.txt"
data_subset = []

In [10]:
from collections import defaultdict
# Read the file and organize categories
data_subset = []
categories = defaultdict(list)
current_category = None
category_order = []  # Maintain order of categories

with open(class_file, "r") as f:
    for line in f:
        line = line.strip()
        if not line or line.startswith("//"):
            if line.startswith("//"):
                #this is category name
                current_category = line.strip("//").strip()
                if current_category not in category_order:
                    category_order.append(current_category)
        else:
            categories[current_category].append(line)
            #the line is the folder name
            data_subset.append(line)

In [11]:
data_subset

['eating burger',
 'eating cake',
 'eating carrots',
 'eating chips',
 'eating doughnuts',
 'eating hotdog',
 'eating ice cream',
 'eating spaghetti',
 'eating watermelon',
 'drinking',
 'drinking beer',
 'drinking shots',
 'tasting beer',
 'tasting food',
 'baking cookies',
 'barbequing',
 'cooking chicken',
 'cooking egg',
 'cooking on campfire',
 'cooking sausages',
 'frying vegetables',
 'making a cake',
 'making a sandwich',
 'making pizza',
 'making sushi',
 'making tea',
 'scrambling eggs',
 'answering questions',
 'counting money',
 'waiting in line',
 'celebrating',
 'hugging',
 'bartending',
 'setting table',
 'counting money',
 'cleaning floor',
 'cleaning shoes',
 'cleaning toilet',
 'cleaning windows',
 'mopping floor',
 'washing dishes',
 'washing hands',
 'carrying baby',
 'pushing cart',
 'picking fruit',
 'unloading truck']

In [12]:
categories

defaultdict(list,
            {'Eating-related': ['eating burger',
              'eating cake',
              'eating carrots',
              'eating chips',
              'eating doughnuts',
              'eating hotdog',
              'eating ice cream',
              'eating spaghetti',
              'eating watermelon'],
             'Drinking-related': ['drinking',
              'drinking beer',
              'drinking shots',
              'tasting beer',
              'tasting food'],
             'Cooking-related': ['baking cookies',
              'barbequing',
              'cooking chicken',
              'cooking egg',
              'cooking on campfire',
              'cooking sausages',
              'frying vegetables',
              'making a cake',
              'making a sandwich',
              'making pizza',
              'making sushi',
              'making tea',
              'scrambling eggs'],
             'ordering food': ['answering questions',
              

In [15]:
from torch.utils.data import DataLoader, Dataset
import os
from vivit import load_video
#create a custom video dataset from each video in data_subset folder
data_root = './kinetics400_5per'
import torch
class VideoDataset(Dataset):
    def __init__(self, video_paths, labels, num_frames=16, image_size=224):
        self.video_paths = video_paths
        self.labels = labels
        self.num_frames = num_frames
        self.image_size = image_size

    def __len__(self):
        return len(self.video_paths)
    def __getitem__(self, idx):
        video = load_video(os.path.join(data_root, self.video_paths[idx]), num_frames=16, image_size=224).cuda()
        label = torch.tensor(self.labels[idx], dtype=torch.long).cuda()
        return video, label

AttributeError: partially initialized module 'torch' has no attribute 'nn' (most likely due to a circular import)

In [ ]:
videos = []
labels = []
for cls in data_subset:
    for video in os.listdir(os.path.join(data_root, cls)):
        videos.append(video)
        labels.append(cls)

In [ ]:
def train_model(model, dataloader, criterion, optimizer, num_epochs=10, device='cuda'):
    model.train()
    for epoch in range(num_epochs):
        total_loss, correct, total = 0, 0, 0
        for videos, labels in dataloader:
            videos, labels = videos.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(videos)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            correct += (outputs.argmax(dim=1) == labels).sum().item()
            total += labels.size(0)
        
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(dataloader):.4f}, Accuracy: {correct/total:.4f}")